In [1]:
%cd C:\Users\yukir\Documents\GitHub\Scrapper\py

from nlp_processing import *
set_global_determinism(seed=1337)

%cd C:\Users\yukir\Documents\Monicas_workspace\Korea_medicine\LDA

C:\Users\yukir\Documents\GitHub\Scrapper\py
True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2209060646499225746
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9759592966142956445
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9883535296
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13194835299884226662
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4500065859755669378
physical_device_desc: "device: XLA_GPU device"
]
tf 2.2.0
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
C:\Users\yukir\Documents\Monicas_workspace\Korea_medicine\LDA


In [2]:
from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric 
import logging 
import pickle 
import pyLDAvis

from gensim.models.coherencemodel import CoherenceModel 
import matplotlib.pyplot as plt

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

from konlpy.tag import Mecab 
from konlpy.tag import Okt
okt = Okt()
from tqdm import tqdm 
import re 
import pickle 
import csv

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
from gensim.models import CoherenceModel
import gensim
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\scipy\io\matlab\mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5


In [3]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 
    
def get_nouns(tokenizer, sentence): 
    """ 단어의 길이가 2이상인 일반명사(NNG),
     고유명사(NNP), 외국어(SL)만을 반환한다. 
     :param tokenizer: 
     :param sentence: :return: """ 
    tagged = tokenizer.nouns(sentence)
    nouns = [s for s in tagged if len(s)>1] 
     
    return nouns 
     
     
def tokenize(df): 
    tokenizer = okt
    processed_data = [] 
    for sent in tqdm(df['cleaned_review']):
        sentence = clean_text(sent.replace('\n', '').strip()) 
        processed_data.append(get_nouns(tokenizer, sentence)) 
        
    return processed_data
    
    
def save_processed_data(processed_data): 
    """ 토큰 분리한 데이터를 csv로 저장 :param processed_data: :return: """ 
    
    with open('lsa_token_pos.csv', 'w', newline='', encoding='utf-8') as f: 
        writer = csv.writer(f) 
        for data in processed_data:
             writer.writerow(data)


In [4]:
cleaned_nlp = pd.read_csv('cleaned_NLP.csv', index_col =0)
cleaned_nlp.head()
cleaned_nlp = cleaned_nlp.drop_duplicates()
cleaned_nlp = cleaned_nlp.dropna()
df = cleaned_nlp.copy()
df = df[df['stars']<=6.2]
df.head(3)

,hospital,stars,reviews,adress,cleaned_review
1,,,,,
108,삼양으뜸한의원,4.0,목디스크때문에 며칠동안 목이 너무너무 아팠어요 마침 집 앞에 있는 한의원이 있어서...,서울,목 디스크 때문에 며칠 동안 목이 너무너무 아팠어요 마침 집 앞에 있는 한의원이 있...
202,김정호한의원,6.0,지인소개로 갔는데 친절하고 청결해 좋았고 기다리지 않았으며 늦은시간에도 진료할수있어...,충북,지인 소개로 갔는데 친절하고 청결해 좋았고 기다리지 않았으며 늦은 시간에도 진료할 ...
216,동명한방병원,5.5,침치료만 좋아요1진료실에서 침치료할때는 아픈 부위에다가 직접 침 놓으며 안아프고 효...,전남,침 치료만 좋아요 1진료실에서 침 치료할 때는 아픈 부위에다가 직접 침 놓으며 안 ...


In [5]:
if __name__ == '__main__':
     # df = pd.read_csv('negative.csv', index_col = 0)
     processed_data = tokenize(df) # description 부분을 토크나이징 한다. 
     
     # 토큰 분리한 데이터를 저장 
     save_processed_data(processed_data)



100%|██████████| 481/481 [00:07<00:00, 63.68it/s] 


In [6]:
from nltk.corpus import stopwords  
stopwords.words('korean')[:10] 

stopwords_kor = stopwords.words('korean')

## **TF-IDF행렬 만들기**

In [7]:
detokenized_doc = []
for i in range(len(processed_data)):
    t = ' '.join(processed_data[i])
    detokenized_doc.append(t)

df['detokenized_doc'] = detokenized_doc # 다시 text['headline_text']에 재저장

df['detokenized_doc'][:5]

1
108    디스크 때문 며칠 동안 마침 한의원 가게 진료 실과 화장실 대기실 의사 선생님 성의...
202           지인 소개 시간 진료 믿음 진료 자세 윈장님 설명 효과 다시 생각 지인 소개
216    치료 진료 치료 부위 직접 효과 진료 대표 원장 아픔 부위 진짜 효과 시설 노후 물...
219            선생님 설명 치료 과정 안내 과정 효과 다른 치료 문의 중단 물리치료 기억
225    평소 허리 갑자기 한의원 방문 정형외과 물리치료 효과 불구 물리치료 종류 병원 잠깐 효과
Name: detokenized_doc, dtype: object

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words=stopwords_kor,             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[가-힣0-9]{1,}',  # num chars > 3
                             max_features=2000,             # max number of uniq words
                            ) 
data_vectorized = vectorizer.fit_transform(df['detokenized_doc'])
data_vectorized.shape # TF-IDF 행렬의 크기 확인

(481, 175)

In [9]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  6.1051381051381055 %


In [17]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=3,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=1337,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_top=lda_model.fit_transform(data_vectorized)

print(lda_model.components_)
print(lda_model.components_.shape) 

[[ 2.25108547  6.97590694  2.59647954  2.17603212  2.61707437 13.54032781
   4.43578013  5.27245646  2.94474763  0.44026411  2.36247628  0.41069997
   4.77470941  0.40851517  3.36993005  4.19298437  1.46964082  1.71061187
   4.1575211   1.9185619   3.21425397  6.40961016  0.90048683 11.38248177
   2.73631733  3.31144728  9.549399    3.44046976  5.0845856   1.91390119
   4.56008742  6.42988599  0.59539033  2.59798707  8.42496507  0.36196048
   9.39683456  4.48569475  0.75873752  4.83055923  7.77847     2.05887252
   6.65498292  5.72139855  1.65423057  4.40874036  4.89918107  4.27931734
   3.4922759   0.55630067  2.5057768   3.60125782  4.25594749  3.13932144
  20.06516986  0.91493635 25.31266576  2.70294421  7.76038143 27.20577804
   2.23222565  0.90823324  0.77612029  0.47479996  0.6807348   0.46728053
   0.58814509  6.81787471  5.50475627  8.13380538  5.66765017  1.15121323
   4.06498202  3.69613505  0.64919958  0.47313681 18.62439391 12.94995631
   1.01165655 11.63819217  6.27368582 

In [18]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
print(lda_model.get_params())

Log Likelihood:  -7946.699033997285
Perplexity:  281.63352546310836
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'online', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 10, 'mean_change_tol': 0.001, 'n_components': 3, 'n_jobs': -1, 'perp_tol': 0.1, 'random_state': 1337, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


## **GRID SEARCH**

In [24]:
# Define Search Param
search_params = {'n_components': [3, 5, 7, 10], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [3, 5, 7, 10]})

In [25]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.5, 'n_components': 3}
Best Log Likelihood Score:  -1917.6329268049885
Model Perplexity:  282.34220413185625


In [19]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(df['cleaned_review'].tolist()))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,dominant_topic
Doc0,0.820000,0.090000,0.100000,0
Doc1,0.160000,0.670000,0.170000,1
Doc2,0.810000,0.090000,0.100000,0
Doc3,0.580000,0.100000,0.330000,0
Doc4,0.810000,0.090000,0.100000,0
Doc5,0.160000,0.100000,0.750000,2
Doc6,0.240000,0.090000,0.670000,2
Doc7,0.160000,0.120000,0.720000,2
Doc8,0.610000,0.240000,0.140000,0
Doc9,0.280000,0.120000,0.600000,2


In [20]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('치료', 46.16), ('진료', 28.79), ('병원', 27.21), ('방문', 25.31), ('물리치료', 20.07), ('의사', 19.0), ('효과', 18.77), ('선생님', 18.62), ('한의원', 17.52), ('원장', 15.72)]
Topic 2: [('지인', 7.68), ('체질', 7.45), ('진단', 5.59), ('상담', 5.22), ('한약', 4.3), ('추천', 3.97), ('후기', 3.67), ('보험', 3.49), ('건강', 3.26), ('소개', 3.23)]
Topic 3: [('진료', 17.13), ('환자', 11.41), ('보고', 10.57), ('다이어트', 10.54), ('설명', 10.34), ('직원', 10.25), ('효과', 9.85), ('상담', 9.4), ('병원', 9.09), ('처방', 8.84)]


In [12]:
# dictionary = corpora.Dictionary(processed_data) 
# dictionary.filter_extremes(no_below=10, no_above=0.05) 
# corpus = [dictionary.doc2bow(text) for text in processed_data] 

In [21]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

C:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \
C:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\sklearn\manifold\_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=                 x            y  topics  cluster       Freq
topic                                                      
0     -1015.700562  1338.981201       1        1  53.533677
2      -387.996002  -373.316162       2        1  32.688127
1     -2184.745605   -60.565800       3        1  13.778196, topic_info=    Term       Freq      Total Category  logprob  loglift
128   지인  10.000000  10.000000  Default  30.0000  30.0000
144   체질   9.000000   9.000000  Default  29.0000  29.0000
131   진단   7.000000   7.000000  Default  28.0000  28.0000
73    상담  17.000000  17.000000  Default  27.0000  27.0000
164   한약  19.000000  19.000000  Default  26.0000  26.0000
..   ...        ...        ...      ...      ...      ...
56    방문   2.769426  30.948313   Topic3  -4.2498  -0.4316
132   진료   2.891902  42.497693   Topic3  -4.2065  -0.7055
40    대기   1.847129  15.636797   Topic3  -4.6548  -0.1539
91    예약   1.707430  14.243662   Topic3  -4.7334  -0.1392
67    부분   1.544362   8.710720   Topic3  -4.8338   0.2521

[166 rows x 6 columns], token_table=      Topic      Freq Term
term                      
2         1  0.466964   가기
2         2  0.233482   가기
2         3  0.233482   가기
3         1  0.446999   가면
3         2  0.223500   가면
...     ...       ...  ...
173       1  0.632223   효과
173       2  0.355626   효과
173       3  0.039514   효과
174       2  0.168931   후기
174       3  0.675726   후기

[249 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

In [22]:
df_document_topic.to_csv('LDA_tfidf_neg.csv')

In [23]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,0,308
1,2,146
2,1,27


In [24]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,가게,가격,가기,가면,가장,간호사,갑자기,개선,개월,개인,거리,건강,건물,검사,검색,결과,결제,경험,계속,과잉,관리,교통사고,권유,그냥,그다지,그대로,근처,금액,기계,기본,기분,기억,나름,나이,내부,내용,느낌,다리,다이어트,달라,대기,대답,대충,대한,데스크,동네,디스크,리뷰,마사지,마음,...,주사,주차,증상,지인,직원,직접,진단,진료,진맥,진짜,진찰,진행,질문,찜질,차도,참고,처방,처음,처치,체질,초진,추가,추나,추천,치료,친절,침대,침술,카운터,코로나,크게,태도,통증,편입,평일,피부,하니,하루,한번,한약,한의사,한의원,항상,해주시,허리,호전,확인,환자,효과,후기
Topic0,2.251085,6.975907,2.596480,2.176032,2.617074,13.540328,4.435780,5.272456,2.944748,0.440264,2.362476,0.410700,4.774709,0.408515,3.369930,4.192984,1.469641,1.710612,4.157521,1.918562,3.214254,6.409610,0.900487,11.382482,2.736317,3.311447,9.549399,3.44047,5.084586,1.913901,4.560087,6.429886,0.595390,2.597987,8.424965,0.361960,9.396835,4.485695,0.758738,4.830559,7.778470,2.058873,6.654983,5.721399,1.654231,4.408740,4.899181,4.279317,3.492276,0.556301,...,1.937498,1.470529,11.350602,1.089111,11.814502,2.318539,0.423081,28.793370,1.229116,6.642920,1.520622,4.637546,0.827928,5.883413,0.979075,1.745577,2.184417,12.120356,4.177143,0.531449,1.547239,1.139453,11.283218,8.333077,46.164056,3.496923,5.118216,6.282452,2.328973,2.512167,1.627018,1.146649,13.290527,3.461084,2.343936,7.098554,3.029510,1.545321,4.677341,8.696442,2.717168,17.517086,4.627993,3.791303,14.264699,6.048733,2.281397,10.092916,18.774439,0.360933
Topic1,0.360423,0.407587,0.833811,1.122183,0.390496,0.414893,0.365548,0.549911,1.097880,0.477065,1.236842,3.260305,1.910659,0.414107,0.715799,1.070580,1.309023,0.380969,1.148264,0.403364,0.373657,0.374154,0.583174,0.362030,0.365587,0.355014,0.356644,0.37436,0.360069,0.360321,1.124351,0.363958,0.467586,0.424017,0.399212,0.386340,0.885856,0.356434,0.382338,0.504666,1.528641,0.365301,0.355038,0.446926,0.514825,0.486296,0.352206,0.646235,0.403438,0.698444,...,0.393316,2.614244,0.702747,7.677789,0.409687,0.501815,5.591233,2.393271,1.410646,0.358491,2.353509,1.076013,0.360239,0.693777,0.761765,0.368392,0.593931,0.410398,0.356416,7.451901,2.605329,0.359505,0.455424,3.965678,0.806053,0.460370,0.539055,0.654548,0.366988,0.360821,1.346498,0.403456,0.729867,0.360921,0.365125,0.388551,1.250769,0.443982,0.378125,4.295636,0.526130,2.814164,0.377290,0.360509,0.469206,0.364527,2.657705,0.839028,0.565819,3.669262
Topic2,1.571718,4.653723,1.220475,1.431472,1.251116,4.271986,0.609044,0.354584,1.867558,4.799561,1.778065,0.467800,0.389213,8.480935,0.438019,0.724557,3.307774,2.374844,4.099091,1.858458,0.589223,0.349024,4.141091,6.759447,2.999932,0.509061,0.471028,2.27774,2.323547,3.017351,4.270567,1.717473,4.683327,1.230212,0.833554,3.319338,7.807349,0.388567,10.537444,0.770818,8.010531,2.949912,1.129405,1.791564,4.297339,4.159235,0.356678,1.493007,0.595641,6.143928,...,4.727404,1.392545,1.339904,0.867505,10.252608,1.062111,0.543237,17.128688,1.972087,0.559414,2.116285,1.409802,3.867530,0.955888,3.577532,1.869903,8.841906,2.015130,0.357628,0.402049,0.542950,4.412113,0.407451,5.866396,6.308815,2.159961,0.491866,0.597297,1.006601,1.215988,1.494470,5.029414,0.483856,0.398059,2.061361,0.457703,1.445958,1.973921,1.812273,7.642824,5.184452,6.529302,0.418701,1.512045,1.405234,0.449035,0.402607,11.408955,9.853983,1.297867


In [25]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,치료,진료,병원,방문,물리치료,의사,효과,선생님,한의원,원장,허리,간호사,통증,의원,설명
Topic 1,지인,체질,진단,상담,한약,추천,후기,보험,건강,소개,한의원,확인,주차,초진,보통
Topic 2,진료,환자,보고,다이어트,설명,직원,효과,상담,병원,처방,원장,검사,선생님,대기,느낌
